In [21]:
################################################################################
#
# Licensed Materials - Property of IBM
# (C) Copyright IBM Corp. 2019
# US Government Users Restricted Rights - Use, duplication disclosure restricted
# by GSA ADP Schedule Contract with IBM Corp.
#
################################################################################


In [22]:
# determine if autoai_libs needs to be updated in the service pod, and if so, install the minimum required version
import subprocess
target_package = 'autoai-libs=='
process = subprocess.run('pip freeze | grep ' + target_package, stdout=subprocess.PIPE, shell=True)
output = process.stdout.decode('ascii')
installed_vrm = ['0','0','0']
if output.startswith(target_package):
    version_string = output[len(target_package):-1]  # remove newline from end too
    installed_vrm = version_string.split('.')
minimum_vrm = ['1','10','3']
update_libs = False
for i, val in enumerate(installed_vrm):
    if int(val) < int(minimum_vrm[i]):
        update_libs = True
        break
if update_libs:
    module_name = '/opt/ibm/.autoAI/autoai_libs-1.10.3-1-cp36-cp36m-linux_x86_64.whl'
    print('attempting pip install of ' + module_name)
    process = subprocess.Popen('pip install ' + module_name, shell=True)
    process.wait()
    print('installation successful')

## IBM AutoAI Auto-Generated Notebook v1.10.5
### Representing Pipeline: P2 from run b6217b47-ee64-415d-9ef3-96a89497079d

**Note**: Notebook code generated using AutoAI will execute successfully.  If code is modified or reordered, there is no guarantee it will successfully execute, please read our documentation for more information https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .

Before modifying the pipeline or trying to re-fit the pipeline, consider:
The notebook converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).
The known_values_list is passed by reference and populated with categorical values during fit of the preprocessing pipeline.  Delete its members before re-fitting.


### 1. Set Up

In [23]:
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import nan, dtype, mean
import autoai_libs
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]


### 2. Compose Pipeline

In [24]:
# The code was removed by Watson Studio for sharing.

Pipeline successfully instantiated


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [25]:
# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
data_provenance = _input_metadata['data_provenance']


In [26]:
import pandas as pd
df = None
if type(data_provenance) is str:
    try:
        df = pd.read_csv(data_provenance, encoding = 'unicode_escape') # your data file name here
    except Exception as e:
        print(e)
if df is None:
    try:
        data_location = data_provenance['input_data'][0]
        print('data_location '+ str(data_location))
        import boto3
        session = boto3.session.Session()
        cos = session.client(
            service_name='s3',
            aws_access_key_id=data_location['connection']['access_key_id'],
            aws_secret_access_key=data_location['connection']['secret_access_key'],
            endpoint_url=data_location['connection']['endpoint_url'],
            verify=False
        )
        local_path = data_location['location']['path']
        print('local_path ' + str(local_path))
        cos.download_file(data_location['location']['bucket'],
                     data_location['location']['path'],
                     local_path)
        df = pd.read_csv(local_path,encoding = 'unicode_escape') # your data file name here
    except Exception as e:
        print(e)
if df is None:
    raise(ValueError('need location or credential information to read dataframe from COS'))


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data_location {'id': '1', 'type': 's3', 'connection': {'access_key_id': 'dc27e481d1284502bd429d300152ac21', 'secret_access_key': 'd721e2ce1e264176477d0c4688d3922b144dc8c0817c05ba', 'endpoint_url': 'https://s3-api.us-geo.objectstorage.softlayer.net'}, 'location': {'type': 's3', 'bucket': 'quotas-donotdelete-pr-6agytp5db6tigo', 'path': 'pipeline_transformed.csv'}}
local_path pipeline_transformed.csv


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [27]:
target = target_label_name # your target name here
df_y = df[target]


In [28]:
df_X = df.drop(columns=[target])

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
# until the problem type is available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
from sklearn.utils.multiclass import type_of_target
if type_of_target(df_y.values) in ['multiclass', 'binary']:
    X, X_holdout, y, y_holdout = train_test_split(df_X.values, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(df_X.values, df_y.values, test_size=holdout_fraction, random_state=random_state)


In [31]:
pipeline.fit(X, y)


Pipeline(memory=None,
     steps=[('column_selector', ColumnSelector(activate_flag=True,
        columns_indices_list=[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25])), ('preprocessor', Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('categ...nt=True,
        subsample=0.9735262976186015, subsample_for_bin=200000,
        subsample_freq=4))])

In [32]:
y_pred = pipeline.predict(X_holdout)


In [33]:
print(y_holdout)
print(y_pred)

[False  True  True ... False  True  True]
[False False  True ... False  True False]


In [51]:
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
print( scorer)
scorer(pipeline, X_holdout, y_holdout)


make_scorer(roc_auc_score, needs_threshold=True)


ValueError: Found unknown categories [4] in column 4 during transform

In [48]:
print (y_holdout)
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_holdout, y_pred)
print("ROC on Test :" + str(auc))

[False  True  True ... False  True  True]
ROC on Test :0.7654619706340589


In [35]:
#delete CONTENTS of known_values_list before refitting, cloning or cross_validate-ing the pipeline, or previous values will be used.
for i in range(len(known_values_list)):
    del known_values_list[0]

In [36]:
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
preprocessor = pipeline.steps[0][1]
preprocessor.fit(X)
X_prep = preprocessor.transform(X)

In [37]:
# remove preprocessor from pipeline
del pipeline.steps[0]

In [38]:
# cross_validate on remaining steps of pipeline
from sklearn.model_selection import cross_validate
cv_results = cross_validate(pipeline, X_prep, y, scoring={optimization_metric:scorer})
import numpy as np
np.mean(cv_results['test_' + optimization_metric])


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: C

0.8238342915877607

In [39]:
cv_results

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('train_roc_auc'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


{'fit_time': array([40.48626971, 37.33702946, 39.4433763 ]),
 'score_time': array([0.54082561, 0.49911213, 0.59702539]),
 'test_roc_auc': array([0.81877205, 0.82879005, 0.82394078]),
 'train_roc_auc': array([1., 1., 1.])}